In [1]:
from regelum.scenario import Scenario
from regelum.simulator import CasADi
import numpy as np
from tqdm.notebook import tqdm

import sys
sys.path.append('../')

from src.system import HydraulicSystem
# from src.policy import PDController

In [2]:
from regelum.policy import Policy
from regelum.utils import rg
from regelum.model import ModelWeightContainer
from regelum.optimizable.core.configs import CasadiOptimizerConfig
from numpy import ndarray
from regelum.model import Model, ModelNN, ModelQuadLin
from regelum.optimizable.core.configs import OptimizerConfig
from regelum.objective import mpc_objective
from regelum.predictor import EulerPredictor
from regelum.objective import RunningObjective
from regelum.data_buffers import DataBuffer

In [3]:
from regelum.callback import (
    ScenarioStepLogger,
    # StateTracker,
    # HistoricalDataCallback,
)
from src.callback import HistoricalDataCallback, SimulatorStepLogger

from regelum import set_ipython_env
%matplotlib inline
callbacks = [SimulatorStepLogger, ScenarioStepLogger, HistoricalDataCallback]
ScenarioStepLogger.cooldown = 0.01
callbacks = set_ipython_env(callbacks=callbacks, interactive=True)

In [ ]:
MAX_STEP = 1e-7
PREDICTION_HORIZON = 10
FINAL_TIME = 10e-3
SAMPLING_TIME = 1e-3
ACTION_SIZE = 1

pred_step_size = 3 * SAMPLING_TIME

# Define the initial state (initial position of the kinematic point).
p_atm = 1e5
initial_state = np.array([1e3, 0, 0, p_atm, p_atm])

model = ModelWeightContainer(
    dim_output=ACTION_SIZE, 
    weights_init=rg.zeros((PREDICTION_HORIZON + 1, ACTION_SIZE), rc_type=rg.CASADI)
)  # A trivial model contains our predicted actions
system = HydraulicSystem(
    init_state=initial_state
)
predictor = EulerPredictor(
    system=system, pred_step_size=pred_step_size
)  # Predictor is used for
# RUNNING OBJECTIVE